# 1. 문제정의

## 다음 분기에 어떤 게임을 설계할지 결정하시오.

In [85]:
import pandas as pd
import matplotlib.pyplot as plt

In [86]:
df = pd.read_csv('vgames2.csv', index_col=0) # 엑셀 CSV이기에 인덱스 열 제거
df.head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


---
---

# 2. 데이터 전처리 (16,598 * 9 -> 11,240 * 11)

## 행 일괄 처리(16,598 -> 16,241)

In [87]:
# 완전 중복 값만 제거 - 동일 게임명이지만 다른 플랫폼의 중복값 처리는 배제함 (16,598 -> 16,597)
df = df.drop_duplicates()

In [88]:
# Nan Data 제거 - 대체 값 삽입으로 오류를 만들 수 있기에 배제함 (16,597 -> 16,241)
df = df.dropna() # 356(Year=270, Genre=50, Publisher=58)개 삭제 - df.isnull().sum()로 확인

## 열별 전처리 (16,241 * 9 -> 11,240 * 11)

In [89]:
# Name(이름) - '.hack//' 8개 오류 변경, 나머지 변경은 도메인 부족으로 후속검증 조치
df.Name = df.Name.str.replace('.hack//','', regex=False)

In [90]:
# Platform(플랫폼) - 오류 및 기타 사유 값은 분석에서 제외함 (16,241 -> 13,129)
  # 오류 값 - '2600' : 대체 값이 없고, 연도와는 무관해 보이는 값
  # 기타사유 - 'PCFX' etc : 특정 국가에서만 판매, 생산량 100개 이하, 10년 내 미생산은 단종으로 처리

df = df[ df['Platform'] != '2600' ] # 101개 데이터
df = df[ (df['Platform'] != 'PCFX') & (df['Platform'] != 'TG16') & (df['Platform'] != '3DO') & (df['Platform'] != 'SCD') & (df['Platform'] != 'WS') &
         (df['Platform'] != 'GEN') & (df['Platform'] != 'DC') & (df['Platform'] != 'NES') & (df['Platform'] != 'GB') & (df['Platform'] != 'NG') &
         (df['Platform'] != 'SNES') & (df['Platform'] != 'SAT') &
         (df['Platform'] != 'GC') & (df['Platform'] != 'N64') &
         (df['Platform'] != 'Wii') & (df['Platform'] != 'WiiU') ]

In [91]:
# Year(연도) - 기존 df.Year로 데이터 변경 시 'SettingWithCopyWarning' 에러 발생 ==> 추천대로 copy와 loc을 사용
  # 에러 이유: 자료구조인 DataFrame의 Year의 부분 변경이기에 원본과 파생의 수정적 오류를 막기 위한 경고

# 1. 데이터 타입 변경: Float -> Int // 변경 근거: (대소, 범위)연산 진행을 위해 변환
df.Year = df.Year.astype(int)

# 2. 데이터 값 변경: 100이하 값( 0~16= 2000년도 / 86~98= 1900년도 ) // 변경 근거: 몇 개의 게임을 검색해본 결과 변경연도와 일치함
under20 = df[df.Year < 20].copy()
under20['Year'] += 2000
df.loc[under20.index, 'Year'] = under20['Year']

upper85 = df[(df.Year > 85) & (df.Year < 100)].copy()
upper85['Year'] += 1900
df.loc[upper85.index, 'Year'] = upper85['Year']

In [92]:
# Genre(장르) - 오류 2종(Misc, Platform) 데이터 2,554개 제거 (13,129 -> 11,240)
  # Misc(기타, 1,681개) 값은 추후 장르 분석에서 명확한 구분이 되지 않는 데이터로 제거함
  # Platform(873개) 값은 플랫폼 열에는 값이 있기에 잘못 기재한 오류데이터로 제거함
df = df[ (df['Genre'] != 'Misc') & (df['Genre'] != 'Platform') ]

In [93]:
# Sales(출고량) - 수치 연산을 위한 데이터 타입 변경과 열 추가

# 1-1. 소수점을 없애기 위해 최솟값을 1개로 설정하기 위해, 전체 값에 100을 곱한다.
# 1-2. 출고량에 K,M 문자는 K는 10을, M은 10,000을 추가로 곱하고, 문자는 제거한다.
# 2. 데이터 타입 변경: Object -> Int // 변경 근거: (대소, 범위)연산 진행을 위해 변환
bm = df.NA_Sales.str.extract('(\d+M$)').dropna() # 북미 출고량 조정
bk = df.NA_Sales.str.extract('(\d+K$)').dropna()
df.NA_Sales = df.NA_Sales.str.replace('K','').str.replace('M','').astype(float);
df.NA_Sales *= 100;

NA_Salesbk = df.NA_Sales[bk.index].copy()
NA_Salesbk *= 10
df.loc[NA_Salesbk.index, 'NA_Sales'] = NA_Salesbk

NA_Salesbm = df.NA_Sales[bm.index].copy()
NA_Salesbm *= 10000
df.loc[NA_Salesbm.index, 'NA_Sales'] = NA_Salesbm

df.NA_Sales = df.NA_Sales.astype(int)

cm = df.EU_Sales.str.extract('(\d+M$)').dropna() # EU 출고량 조정
ck = df.EU_Sales.str.extract('(\d+K$)').dropna()
df.EU_Sales = df.EU_Sales.str.replace('K','').str.replace('M','').astype(float)
df.EU_Sales *= 100;

EU_Salesck = df.EU_Sales[ck.index].copy()
EU_Salesck *= 10
df.loc[EU_Salesck.index, 'EU_Sales'] = EU_Salesck

EU_Salescm = df.EU_Sales[cm.index].copy()
EU_Salescm *= 10000
df.loc[EU_Salescm.index, 'EU_Sales'] = EU_Salescm

df.EU_Sales = df.EU_Sales.astype(int)

dm = df.JP_Sales.str.extract('(\d+M$)').dropna() # 일본 출고량 조정
dk = df.JP_Sales.str.extract('(\d+K$)').dropna()
df.JP_Sales = df.JP_Sales.str.replace('K','').str.replace('M','').astype(float)
df.JP_Sales *= 1000;

JP_Salesdk = df.JP_Sales[dk.index].copy()
JP_Salesdk *= 10
df.loc[JP_Salesdk.index, 'JP_Sales'] = JP_Salesdk

JP_Salesdm = df.JP_Sales[cm.index].copy()
JP_Salesdm *= 10000
df.loc[JP_Salesdm.index, 'JP_Sales'] = JP_Salesdm

df.JP_Sales = df.JP_Sales.astype(int)

em = df.Other_Sales.str.extract('(\d+M$)').dropna() # 기타 출고량 조정
ek = df.Other_Sales.str.extract('(\d+K$)').dropna()
df.Other_Sales = df.Other_Sales.str.replace('K','').str.replace('M','').astype(float)
df.Other_Sales *= 1000;

Other_Salesek = df.Other_Sales[dk.index].copy()
Other_Salesek *= 10
df.loc[Other_Salesek.index, 'Other_Sales'] = Other_Salesek

Other_Salesem = df.Other_Sales[cm.index].copy()
Other_Salesem *= 10000
df.loc[Other_Salesem.index, 'Other_Sales'] = Other_Salesem

df.Other_Sales = df.Other_Sales.astype(int)


# 3. ['전체 출고량'] 열(개별 출고량 합) 과 ['Platform_num'] 열(플랫폼을 수치화) 추가
df['Total_Sales'] = df.NA_Sales + df.EU_Sales + df.JP_Sales + df.Other_Sales

a , b = df.Genre.value_counts().index , df.Platform.value_counts().index; rea , reb = {} , {}; list1 , list2 = df.Genre , df.Platform
for i in range(len(b)): # 플랫폼 수치화
  emp = b[i]; reb[emp]=i+1
for key , value in reb.items(): # ['Platform_num'] 열을 추가
  list2 = list2.replace(key, value)
df['Platform_num'] = list2

In [96]:
df = df[ ['Name' , 'Platform' , 'Platform_num' , 'Year' , 'Genre', 'Publisher' , 'NA_Sales' , 'EU_Sales' , 'JP_Sales' , 'Other_Sales' , 'Total_Sales' ] ]
df.head(3)

,Name,Platform,Platform_num,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
1,Candace Kane's Candy Factory,DS,2,2008,Action,Destineer,4,0,0,0,4
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,5,2010,Adventure,Alchemist,0,0,20,0,20
6,Power Pro Kun Pocket 8,DS,2,2005,Sports,Konami Digital Entertainment,0,0,140,0,140


In [ ]:
# 데이터 전처리 종료 - 11,240개의 데이터를 모집단으로 재설정
df.to_csv('전처리완료.csv', index=False)

---
---

# 3. 데이터 분석 및 시각화

## 데이터 정의 - 중점 분석 데이터 = 수치(연도&판매량)와 플랫폼&장르 데이터

### **Year(연도)** : 1985 ~ 2020년
> 연도로써 최근 10년의 의미는 2006~2016년으로 정의함 <br>

> 이유: 2020, 2017년을 분석에 포함하기에는 각각 1, 3개의 데이터로 빈약하고 <br>
> 2019 ~ 2018년의 데이터가 없기에 2016년부터를 기준으로 함

In [ ]:
# 최고 매출 플랫폼 연도 분석
aa = df [ (df.Platform == 'X360') & (df.Year != 2005) ]; bb = df [ df.Platform == 'PS3' ]

aax = aa.groupby(['Year'])['Total_Sales'].sum().reset_index(); bbx = bb.groupby(['Year'])['Total_Sales'].sum().reset_index()
adx = aax.Total_Sales.mean(); bdx = bbx.Total_Sales.mean()

plt.figure( figsize=(20,10) ); plt.xticks(aax.Year);
plt.bar(aax.Year , aax.Total_Sales , label='X360 Mean = 3,653,464', color='red' , alpha=0.4 );
plt.bar(bbx.Year , bbx.Total_Sales , label='Ps3 Mean = 2,700,813', color='blue' , alpha=0.4 );
plt.axhline(y=adx, color='r', linewidth=1); plt.axhline(y=bdx, color='blue', linewidth=1)

for i in range(len(bbx)):
    height = aax.Total_Sales[i]
    plt.text(aax.Year[i], height + 100, '%.0f' %height, ha='center', va='bottom', size = 12)

    he = bbx.Total_Sales[i]
    plt.text(bbx.Year[i], he + 100, '%.0f' %he, ha='center', va='bottom', size = 12)
    
plt.legend(fontsize=25);

In [ ]:
aa = df [ (df.Platform == 'X360') & (df.Year != 2005) & (df.Genre != 'Simulation') ];
bb = df [ (df.Platform == 'PS3') & (df.Genre != 'Sports') & (df.Genre != 'Fighting') & (df.Genre != 'Simulation') ]

aax = aa.groupby(['Year'])['Total_Sales'].transform(max) == aa['Total_Sales']; bbx = bb.groupby(['Year'])['Total_Sales'].transform(max) == bb['Total_Sales']
aab = list(aa[aax].Year); aac = list(aa[aax].Genre); bbb = list(bb[bbx].Year); bbc = list(bb[bbx].Genre)
abx = pd.DataFrame( (zip(aab, aac)) , columns=['year' , 'genre']); bbx = pd.DataFrame( (zip(bbb, bbc)) , columns=['year' , 'genre']);
abx = abx.sort_values('year') ; bbx = bbx.sort_values('year')

plt.figure( figsize=(20,10) ); plt.xticks(abx.year);
plt.scatter(abx.year , abx.genre, color='red' , label='X360 - 4Shooter, 4Action', marker='x' , s=350 );
plt.scatter(bbx.year , bbx.genre, color='blue' , label='PS3 - 3Role-Playing', marker='o' , s=350 );
plt.legend(fontsize = 30);

### **Salse - each & Total**
출고량은 소비자의 구매량이므로 출고량 0은 무료게임이 아닌 판매가 안 된 제품임 <br> 출고량 1의 의미는 1개라는 의미로 해석함

In [ ]:
# 최고 매출량
df.loc[ df.NA_Sales.idxmax() ] # 북미 최고 매출 게임 - Grand Theft Auto V(2013, 액션, X360)
df.loc[ df.EU_Sales.idxmax() ] # 유럽 최고 매출 게임 - FIFA 14(2013, 스포츠, X360)
df.loc[ df.JP_Sales.idxmax() ] # 일본 최고 매출 게임 - Pokemon FireRed(2004, 롤플레잉, GBA)
df.loc[ df.Other_Sales.idxmax() ] # 기타 최고 매출 게임 - Fallout3(2008, 롤플레잉, PS3)

In [ ]:
# 플랫폼 총 매출량
PS_Series = df[df.Platform == 'PS'].Total_Sales.sum() + df[df.Platform == 'PS2'].Total_Sales.sum() + df[df.Platform == 'PS3'].Total_Sales.sum() + df[df.Platform == 'PSP'].Total_Sales.sum() + df[df.Platform == 'PS4'].Total_Sales.sum() + df[df.Platform == 'PSV'].Total_Sales.sum()
Xbox_Series = df[df.Platform == 'XB'].Total_Sales.sum() + df[df.Platform == 'X360'].Total_Sales.sum() + df[df.Platform == 'XOne'].Total_Sales.sum()
Nin_Series = df[df.Platform == 'DS'].Total_Sales.sum() + df[df.Platform == '3DS'].Total_Sales.sum()

platform = [PS_Series , Xbox_Series , Nin_Series , df[df.Platform == 'PC'].Total_Sales.sum() , df[df.Platform == 'GBA'].Total_Sales.sum() ]
labels = ['PS_Series' , 'Xbox_Series' , 'Nin_Series' , 'PC' , 'GBA']

plt.pie(platform, labels=labels , autopct='%.1f%%' );

In [ ]:
# 국가별 판매량 - NA_Sales, EU_Sales, JP_Sales, Other_Sales
PS_Series = df[df.Platform == 'PS'].Other_Sales.sum() + df[df.Platform == 'PS2'].Other_Sales.sum() + df[df.Platform == 'PS3'].Other_Sales.sum() + df[df.Platform == 'PSP'].Other_Sales.sum() + df[df.Platform == 'PS4'].Other_Sales.sum() + df[df.Platform == 'PSV'].Other_Sales.sum()
Xbox_Series = df[df.Platform == 'XB'].Other_Sales.sum() + df[df.Platform == 'X360'].Other_Sales.sum() + df[df.Platform == 'XOne'].Other_Sales.sum()
Nin_Series = df[df.Platform == 'DS'].Other_Sales.sum() + df[df.Platform == '3DS'].Other_Sales.sum()

platform = [PS_Series , Xbox_Series , Nin_Series , df[df.Platform == 'PC'].Other_Sales.sum() , df[df.Platform == 'GBA'].Other_Sales.sum() ]
labels = ['PS_Series' , 'Xbox_Series' , 'Nin_Series' , 'PC' , 'GBA']

plt.pie(platform, labels=labels , autopct='%.1f%%' );

### **Platform(플랫폼)** 남은 13개 세부 그룹화  <BR>
1) 플레이스테이션 시리즈(본체 650,000원) : PS, PS2, PS3&PSP , PS4&PSV <BR>
2) X-Box 시리즈(본체 630,000원) : XB, X360, Xone <BR>
3) 닌텐도 시리즈(본체 380,000원) : DS , 3DS <BR>
4) 기타 : PC , GBA

In [ ]:
# 플랫폼 생산량
PS_Series = len(df[df.Platform == 'PS']) + len(df[df.Platform == 'PS2']) + len(df[df.Platform == 'PS3']) + len(df[df.Platform == 'PSP']) + len(df[df.Platform == 'PS4']) + len(df[df.Platform == 'PSV'])
Xbox_Series = len(df[df.Platform == 'XB']) + len(df[df.Platform == 'X360']) + len(df[df.Platform == 'XOne'])
Nin_Series = len(df[df.Platform == 'DS']) + len(df[df.Platform == '3DS'])

platform_num = [PS_Series , Xbox_Series , Nin_Series , len(df[df.Platform == 'PC']) , len(df[df.Platform == 'GBA']) ]
labels = ['PS_Series' , 'Xbox_Series' , 'Nin_Series' , 'PC' , 'GBA']

plt.pie(platform_num, labels=labels , autopct='%.1f%%' );

### **Genre(장르)** <BR>

In [ ]:
# 장르별 분류
plt.pie(df.Genre.value_counts() , labels = df.Genre.value_counts().index , autopct='%.1f%%' );

In [ ]:
# 국가별 장르 분류
a = df.groupby('Genre').mean()['Total_Sales']; b = df.groupby('Genre').mean()['NA_Sales']; c = df.groupby('Genre').mean()['EU_Sales']
d = df.groupby('Genre').mean()['JP_Sales']; e = df.groupby('Genre').mean()['Other_Sales']
plt.pie(e , labels=e.index , autopct='%.1f%%' );

In [ ]:
# 국가별 장르와 플랫폼
ap = df.groupby('Genre').sum()['Platform_num']

bp = df.groupby( ['Genre' , 'Platform_num'] ).sum()['NA_Sales']
cp = df.groupby( ['Genre' , 'Platform_num'] ).sum()['EU_Sales']
dp = df.groupby( ['Genre' , 'Platform_num'] ).sum()['JP_Sales']
ep = df.groupby( ['Genre' , 'Platform_num'] ).sum()['Other_Sales']

plt.pie(bp , labels=bp.index , autopct='%.1f%%' ); plt.title('NA');

## Index적 역할 데이터 - 이름과 제작사
분석을 통한 결과 값(1~3개, 소수)이 나오면 사후 검증으로 조사함